In [1]:
import duckdb
import pandas as pd
import numpy as np

con = duckdb.connect('slack.duckdb')
pd.set_option('display.float_format', str)

# Fetch key columns from extracted data & build channel message dataframe

In [2]:
messages_df = con.execute("SELECT user, epoch_us(thread_ts)/1000000 as thread_ts, epoch_us(ts) / 1000000 as ts, text FROM slack_export.export_test_message;").fetchdf()

In [3]:
replies_df = con.execute("SELECT user, thread_ts, ts, text FROM slack_export.export_test_replies_message;").fetchdf()

In [4]:
replies_df['thread_ts'] = replies_df['thread_ts'].astype(float)
replies_df['ts'] = replies_df['ts'].astype(float)

In [5]:
df = pd.concat([messages_df, replies_df])
df.sort_values(['thread_ts'], inplace=True)
df['ts'] = pd.to_datetime(df['ts'], unit='us')

In [6]:
df.head(10)

,user,thread_ts,ts,text
1,U08J3DV7SAJ,1742934604.227359,1970-01-01 00:29:02.934604227,Conversation #1 start
2,U08JBSW64Q2,1742934604.227359,1970-01-01 00:29:02.934627187,conversation #1 | response #1
3,U08J3DV7SAJ,1742934604.227359,1970-01-01 00:29:02.934639155,conversation #1 | response #2
0,U08JBSW64Q2,1742934651.309429,1970-01-01 00:29:02.934651309,Conversation #2 start
0,U08J3DV7SAJ,1742934651.309429,1970-01-01 00:29:02.934671323,Conversation #2 | response #1
1,U08JBSW64Q2,1742934651.309429,1970-01-01 00:29:02.934679083,Conversation #2 | response #2


# Parse the messages into "documents"

In [7]:
df['output'] = df.apply(lambda r : f'{r['user']} | {r['ts']} | {r['text']}', 1)

In [8]:
doc_df = df.groupby('thread_ts')['output'].agg('\n'.join).reset_index()

In [9]:
doc_df.to_csv('output.csv', index=False)